In [2]:
import pandas as pd, json, time, sys
from datetime import datetime
from pathlib import Path
from app_store_web_scraper import AppStoreEntry

# ====================================================
# 🪵 LOGGER SETUP — capture stdout to a timestamped log
# ====================================================
BASE = Path("data")
LOG_DIR = BASE / "logs"
LOG_DIR.mkdir(parents=True, exist_ok=True)

timestamp_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_path = LOG_DIR / f"update_log_{timestamp_str}.txt"

class Logger:
    def __init__(self, filename):
        self.terminal = sys.stdout
        self.log = open(filename, "w", encoding="utf-8")
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
    def flush(self):
        self.terminal.flush()
        self.log.flush()

sys.stdout = Logger(log_path)
print(f"🪵 Logging all output to {log_path}\n")

# ====================================================
# CONFIG
# ====================================================
APPS = [
    {"name": "tinder", "id": 547702041},
    {"name": "yubo",   "id": 1038653883},
    {"name": "bumble", "id": 930441707},
    {"name": "meetup", "id": 375990038},
    {"name": "boo", "id": 1498407272},
    {"name": "gofrendly", "id": 1132977013},
]
COUNTRIES = ["fr", "us", "de", "se", "gb", "ca", "it", "es"]

RAW, PROC, META = BASE/"raw", BASE/"processed", BASE/"metadata"
for p in (RAW, PROC, META): p.mkdir(parents=True, exist_ok=True)

LAST_FILE = META/"last_dates.json"
LOG_FILE  = META/"scrape_log.csv"

# ====================================================
# INITIALIZATION
# ====================================================
print("🔧 Setting up environment...\n")
last_dates = json.loads(LAST_FILE.read_text()) if LAST_FILE.exists() else {}
rows_log = []
print(f"📂 Raw path:       {RAW}")
print(f"📂 Processed path: {PROC}")
print(f"📂 Metadata path:  {META}")
print(f"🕓 Last dates loaded for {len(last_dates)} app-country keys.\n")

# ====================================================
# SCRAPE LOOP
# ====================================================
for c in COUNTRIES:
    print(f"\n🌍 COUNTRY: {c.upper()} -------------------------------")

    for app in APPS:
        key = f"{app['name']}_{c}"
        print(f"\n➡️  Fetching {key} ...")
        app_entry = AppStoreEntry(app_id=app["id"], country=c)
        new_rows = []
        newest = None
        oldest = None

        # --- Fetch reviews ---
        try:
            for i, r in enumerate(app_entry.reviews(), start=1):
                d = r.date
                if hasattr(d, "tzinfo") and d.tzinfo is not None:
                    d = d.astimezone(None).replace(tzinfo=None)

                newest = max(newest, d) if newest else d
                oldest = min(oldest, d) if oldest else d

                new_rows.append({
                    "id": r.id,
                    "title": r.title,
                    "rating": r.rating,
                    "date": d,
                    "content": r.content,
                    "country": c,
                    "app": app["name"]
                })

                if i % 100 == 0:
                    print(f"   🔹 {i} reviews fetched so far...")

        except Exception as e:
            print(f"❌ Error fetching {app['name']} ({c}): {e}")
            continue

        if not new_rows:
            print(f"⚠️  No reviews found for {key}. Skipping.")
            continue

        print(f"✅ Finished fetching {len(new_rows)} reviews for {key}")
        raw_name = RAW / f"{key}_{datetime.now().date()}.json"
        pd.DataFrame(new_rows).to_json(raw_name, orient="records", indent=2, force_ascii=False)
        print(f"💾 Saved raw snapshot: {raw_name.name}")

        # --- Merge with master CSV ---
        master_path = PROC / f"master_{key}.csv"
        try:
            df_master = pd.read_csv(master_path)
            print(f"📄 Loaded existing master file ({len(df_master)} rows).")
        except FileNotFoundError:
            df_master = pd.DataFrame()
            print(f"📄 No master file found for {key}. Creating new one...")

        df_new = pd.DataFrame(new_rows)
        if not df_master.empty:
            before = len(df_master)
            df_new = df_new[~df_new["id"].isin(df_master["id"])]
            duplicates = before + len(df_new) - len(df_master)
            print(f"🔍 Found {len(df_new)} new reviews (removed {duplicates} duplicates).")

        df_merged = pd.concat([df_master, df_new], ignore_index=True)
        df_merged.to_csv(master_path, index=False)
        print(f"💾 Master CSV updated: {master_path.name} ({len(df_merged)} total reviews)")

        # --- Update metadata ---
        last_dates[key] = newest.isoformat()
        rows_log.append({
            "timestamp": datetime.now().isoformat(),
            "app": app["name"],
            "country": c,
            "new": len(df_new),
            "total": len(df_merged),
            "newest": newest.isoformat(),
            "oldest": oldest.isoformat()
        })
        print(f"🕒 Latest review date for {key}: {newest.date()} (oldest: {oldest.date()})")

        time.sleep(3)

# ====================================================
# SAVE METADATA & LOGS
# ====================================================
print("\n🗂 Saving metadata and logs...")
with open(LAST_FILE, "w", encoding="utf-8") as f:
    json.dump(last_dates, f, indent=2)
print(f"✅ Updated last_dates.json ({len(last_dates)} keys)")

log_df = pd.DataFrame(rows_log)
if LOG_FILE.exists():
    existing_log = pd.read_csv(LOG_FILE)
    log_df = pd.concat([existing_log, log_df], ignore_index=True)
log_df.to_csv(LOG_FILE, index=False)
print(f"✅ Updated scrape_log.csv ({len(log_df)} total log entries)")

# ====================================================
# SUMMARY TABLE
# ====================================================
print("\n📊 SUMMARY TABLE (this run)")
summary_df = pd.DataFrame(rows_log)[["app", "country", "new", "total", "oldest", "newest"]]
summary_df = summary_df.sort_values(["app", "country"]).reset_index(drop=True)

if not summary_df.empty:
    print(summary_df.to_string(index=False))
else:
    print("⚠️ No new reviews were added in this run.")

# ====================================================
# FINAL
# ====================================================
print(f"\n🪵 Log file saved at: {log_path}")
print("\n🎉 All updates complete!\n")

# Restore stdout (optional cleanup)
sys.stdout.log.close()
sys.stdout = sys.stdout.terminal


🪵 Logging all output to data\logs\update_log_2025-11-03_22-20-38.txt

🔧 Setting up environment...

📂 Raw path:       data\raw
📂 Processed path: data\processed
📂 Metadata path:  data\metadata
🕓 Last dates loaded for 0 app-country keys.


🌍 COUNTRY: FR -------------------------------

➡️  Fetching tinder_fr ...
   🔹 100 reviews fetched so far...
   🔹 200 reviews fetched so far...
   🔹 300 reviews fetched so far...
   🔹 400 reviews fetched so far...
   🔹 500 reviews fetched so far...
✅ Finished fetching 500 reviews for tinder_fr
💾 Saved raw snapshot: tinder_fr_2025-11-03.json
📄 No master file found for tinder_fr. Creating new one...
💾 Master CSV updated: master_tinder_fr.csv (500 total reviews)
🕒 Latest review date for tinder_fr: 2025-11-02 (oldest: 2025-05-18)

➡️  Fetching yubo_fr ...
   🔹 100 reviews fetched so far...
   🔹 200 reviews fetched so far...
   🔹 300 reviews fetched so far...
   🔹 400 reviews fetched so far...
   🔹 500 reviews fetched so far...
✅ Finished fetching 500 review

Ideas:

- show the number of reviews over time of different countries and apps
- do a cron job to update daily each app & country
- work on the preprocess data to clean it, see how to do it per country and then globally
- See to upload it in Supabase
- 